# Reading Dumped data

In [ ]:
with open('sortedFish.pik','rb') as f:
    sortedFish=pickle.load(f)

posU=sortedFish[0]
nonOverlappingRange=sortedFish[1]
fishU=sortedFish[2]

fishU=frame,fish,pixel,[row,col,grey]

# Creating a distance histogram out of the sorted data

In [ ]:
binsDist=np.linspace(0,30,20)
binsDif=np.linspace(0,50,20)
binsSum=np.linspace(150,350,20)
histSum=np.zeros((len(binsDist)-1,len(binsSum)-1))
histDif=np.zeros((len(binsDist)-1,len(binsDif)-1))

pairData=[]
countSum=0
countDif=0
for k in range(*nonOverlappingRange[0]):
    fishA=fishU[k][0]
    for i in range(fishA.shape[0]):
        for j in range(i):
            d=np.sqrt((fishA[i,0]-fishA[j,0])**2+(fishA[i,1]-fishA[j,1])**2)
            bSum=fishA[i,2]+fishA[j,2]
            bDiff=fishA[i,2]-fishA[j,2]
            pairData.append([d,bSum,bDiff])
    pairDataT = np.array(pairData).T
    heightValuesSum,_,_=np.histogram2d(pairDataT[0],pairDataT[1],bins=(binsDist,binsSum))
    histSum+=heightValuesSum
    countSum+=1
    heightValuesDif,_,_=np.histogram2d(pairDataT[0],pairDataT[2],bins=(binsDist,binsDif))
    histDif+=heightValuesDif
    countDif+=1
histSum/=countSum
histDif/=countDif

# do a column map

In [ ]:
fig=plt.figure(figsize=(10,5))

ax = fig.add_subplot(121)
plt.imshow(histSum[:,::-1].T,extent=[binsDist[0], binsDist[-1], binsSum[0], binsSum[-1]],aspect='auto')

ax = fig.add_subplot(122)
plt.imshow(histDif[:,::-1].T,extent=[binsDist[0], binsDist[-1], binsDif[0], binsDif[-1]],aspect='auto')

plt.show()